# GLMs

## What are we talking about?

The following definitions follow [13].

A generalized linear model (GLM) is a linear model ($\eta = x^\top \beta$) wrapped in a transformation (link function) and equipped with a response distribution from an exponential family. The choice of link function and response distribution is very flexible. In a GLM, a predictive distribution for the response variable $Y$ is associated with a vector of observed predictors $x$.  The distribution has the form:

\begin{align}
  p(y \, |\, x)
&=
  m(y, \phi) \exp\left(\frac{\theta\, T(y) - A(\theta)}{\phi}\right)
  & & \textrm{random component / distribution family}
\\
  \theta
&:=
  g(\eta) && \textrm{systematic component / link function}
\\
  \eta
&:=
  x^\top \beta && \textrm{parametric link component / linear predictor}
\end{align}

Here $\beta$ are the parameters; $\phi$ is a parameter representing dispersion ("variance"); and $m$, $T$, $A$ are characterized by the user-specified model family; and $g$ is the **link function.** For background on the exponential family, and how common distributions can be represented in the form of the "random component" above, see [Wikipedia | Exponential family](https://en.wikipedia.org/wiki/Exponential_family#Table_of_distributions).

The expected mean of $Y$ depends on $x$ by composition of **linear response** $\eta$ and (inverse) link function, i.e.:

$$
E[y | \eta] := \mu = g^{-1}(\eta).
$$

Proving this involves use of [moment-generating functions and cumulants](https://en.wikipedia.org/wiki/Exponential_family#Moments_and_cumulants_of_the_sufficient_statistic).

With an offset, we write $\eta = X^T \beta + \gamma$.

## Examples of GLMs

### Normal

Many of the "regression" models we commonly work with are GLMs. For example, the normal distribution with a linear link function is a simple GLM: $y | x \sim N(x^T \beta, \sigma^2)$. If we fit $\beta$ with maximum likelihood, this is least-squares regression. If we replace the linear link function with an exponential link function, we get a simple multiplicative model that is also a GLM: $y | x \sim N(e^{x^T \beta}, \sigma^2)$.


### Tweedie
We usually work with a Tweedie distribution, which is a generalization of Poisson ($p=1$) and Gamma ($p=2$):

\begin{align*}
\theta &= \left\{\begin{array}{ll}
\frac{\mu^{1-p}}{1-p} & \text{if } p \neq 1\\
\log \mu              & \text{if } p = 1
\end{array}\right.,\\
T(y) &= y, \\
A(\theta) &= \left\{\begin{array}{ll}
\frac{\mu^{2-p}}{2-p} & \text{if } p \neq 2\\
\log \mu              & \text{if } p = 2.
\end{array}\right.,
\end{align*}

With $1 < p < 2$, the Tweedie distribution is a [compound Poisson-gamma](https://en.wikipedia.org/wiki/Compound_Poisson_distribution#Compound_Poisson_Gamma_distribution) distribution. $y$ is distributed as if a number $N$ is drawn from a Poisson distribution, and then $N$ draws are taken IID from a gamma distribution and added. This distribution might model the total amount of a claim in an insurance context: There are $N$ incidents, and each incident $i$ has an amount $X_i$, and the total amount is the sum of $X_i$.

# Objective function

To fit a GLM, we minimize the negative log-likelihood (or typically the unit deviance) subject to an elastic net constraint:

$$
\min_{\beta} \mathcal L + \alpha \rho ||\beta||_1  + \frac{\alpha (1-\rho)}{2} ||\beta||_2^2
$$

We might also like to use the grouped lasso, which includes or excludes variables in groups. For example, if we have a categorical variable that we one-hot encode, like the make of a car, we might want to exclude all of the coefficients on that fixed effect together for greater interpretability and ease of use.

# Optimizers
Our objective function will generally be convex, because the log-likelihoods of members of the exponential family are convex in their "natural parameterizations." The natural parameterization may not be the most obvious one. Example: The log-likelihood of the normal distribution is convex in one over the variance, which is its natural parameterization. It is not convex in the variance. We can generally assume we are solving convex problems.

An exception is multicollinearity (rank deficiency) in the design matrix, without an L2 component to the penalty. In that case, the problem will be only weakly convex and will have no unique miminum. This is not an arcane consideration, since we frequently generate rank deficiency by constructing multiple sets of one-hot encoded categorical variables. This can make evaluating different optimizers tricky, since they could converge to different equally good optima. The glmnet paper suggests using at least a small l2 component to remove "degeneracies and wild behavior."

## IRLS
If we don't have a lasso penalty (i.e. $\rho=1$), these problems will furthermore be differentiable. They are usually estimated using Iteratively Reweighted Least Squares (IRLS).

IRLS can be justified in a couple of ways. First, and more intuitively, it can be seen as something something expected mean, something Taylor approximation. This setup works _only_ if the problem is set up as a GLM, and 

Second, the IRLS solution can be seen as taking a Newton step, but replacing the Hessian with the expected Hessian.

Handy equations and derviatives:\begin{align*}
LL &= \sum_i \left( \log(m(y_i, \phi)) + \frac{g(\eta_i) T(y_i) - A(g(\eta_i))}{\phi} \right) \\
LL_i'(\eta_i) &= \frac{ g'(\eta_i) \left( T(y_i) - A'(g(\eta_i)) \right)}{\phi} \\
\frac{\partial LL}{\partial \beta_k} &= \sum_i LL_i' \frac{\partial \eta_i}{\partial \beta_k} \\
&= \sum_i LL_i' x_{ik}
\end{align*}

\begin{align*}
LL(\tilde{\beta}) &\approx LL(\beta) + \sum_i \left( LL_i'(\eta_i) x_i (\tilde{\beta} - \beta) + 
\frac{1}{2} (x_i (\tilde{\beta} - \beta))^2 LL_i''(\eta_i)
\right) \\
 &= f(\beta) + \sum_i \left(
 \frac{1}{2} (x_i \tilde{\beta})^2 LL_i''
 + LL_i' x_i \tilde{\beta} 
- (x_i \tilde{\beta}) LL_i'' x_i \beta
\right) \\
&= f(\beta) + \sum_i \left(
 \frac{1}{2} (x_i \tilde{\beta})^2 LL_i''
 + x_i \tilde{\beta}  \left( 
     LL_i' 
    - LL_i'' x_i \beta
    \right)
\right) \\
&= f(\beta) + \sum_i \frac{1}{2} LL_i'' \left(
 (x_i \tilde{\beta})^2 
 - 2 x_i \tilde{\beta}  \left( 
   1-  \frac{LL_i' }{LL_i''}
    x_i \beta \right)
\right)
\end{align*}

Letting $w_i = \frac{1}{2} \frac{\partial^2 LL_i}{\eta_i^2}$ and
$z_i = 1 - \frac{\frac{\partial LL_i}{\partial \eta_i}}{\frac{\partial LL_i^2}{\partial \eta_i^2}} x_i \beta$,
we can rewrite this as $LL(\tilde{\beta}) \approx f(\beta) + \sum_i w_i \left( x_i \tilde{\beta} - z_i \right)^2$,
and solve this like a least squares problem.

Note that the last term  equals $\tilde{\beta}^T x^T \frac{\partial LL}{\partial \beta} X \beta$, and will be close to zero near the optimum since the gradient is zero near the optimum.

If we want to write terms with $\beta$ in terms of $\eta$,
\begin{align*}
LL(\tilde{\beta}) &\approx f(\beta) + \sum_i \left(
LL_i' x_i \tilde{\beta} 
+ \frac{1}{2} LL_i'' \left(
(x_i \tilde{\beta})^2 - 2 x_i \tilde{\beta} (\eta_i - \gamma_i)
\right)
\right) \\
&= f(\beta) + \sum_i \frac{1}{2} LL_i'' \left(
2 \frac{LL_i'}{LL_i''} \left( x_i \tilde{\beta} \right)
+ 
(x_i \tilde{\beta})^2 - 2 x_i \tilde{\beta} (\eta_i - \gamma_i)
 \right) \\
 &= f(\beta) + \sum_i \frac{1}{2} LL_i'' \left(
(x_i \tilde{\beta})^2 - 2 x_i \tilde{\beta} 
\left( \eta_i - \gamma_i 
- \frac{LL_i'}{LL_i''}  \right)
 \right) \\
  &= f(\beta) + \sum_i \frac{1}{2} LL_i'' \left(
x_i \tilde{\beta} -  
\left( \eta_i - \gamma_i 
- \frac{LL_i'}{LL_i''}  \right)
 \right)^2 \\
\end{align*}

What if we don't have any current parameters? If current parameters are zero, this becomes

### Coordinate Descent
With Lasso constraint, some version of coordinate descent is used for the optimization. Other optimization approaches exist (e.g. cvxpy), but we haven't seen these used in large-scale applications. The classic reference here is the "glmnet paper" [2].

Coordinate Descent is older than the glmnet paper, and is a simple idea. In a problem with data $y$ and $x$ and parameters "params", Coordinate Descent involves repeatedly optimizing one or several parameters while holding the rest fixed.

Interestingly, Wikipedia [claims](https://en.wikipedia.org/wiki/Coordinate_descent#Applications) that Coordinate Descent is often overlooked among researchers because it is simple to implement, and they would rather work on something more interesting.

In [1]:
"""
Cyclical coordinate descent with Newton-like steps and a
twice-differentiable objective function.
"""
def coordinate_descent_inner(y, x, params, grad_func, hess_func):
    for i, elt in enumerate(params):
        step = -grad_func(y, x, params) / hess_func(y, x, params)
        params[i] += step
    return params
        
def coordinate_descent(y, x, params, grad_func, hess_func, 
                       convergence_func):
    while not convergence_func():
        params = coordinate_descent_inner(y, x, params, grad_func,
                                         hess_func)
    return params

This is a bit more complicated when the objective function is not differentiable.

## Features

For our use case in insurance, the critial features for GLMs to be useful for us are
- Need to support Gamma, Poisson (pyglmnet, sklearn-fork, sklearn-master, h20, tfp, statsmodels)
- Need to support sample weights (glmnet, sklearn-fork, sklearn-master, h2ostatsmodels)
- Need to support offsets (an offset is a variable for which we force the parameter to equal 1). (glmnet, h2o, statsmodels)
- Need to work on large-ish data sets (25 million rows, up to several hundred columns)
- Can deal with high-dimensional categorical variables (either through sparse matrices or a dedicated solution) (sparse support: glmnet, h2o, tfp)
- Need to be otherwise sensible (e.g. do not regularize the intercept, etc)
- Decent performance in terms of CPU and memory (commercial competitor without regularization needs a couple of minutes on laptop)
- Reliably convergent algorithms (_not_ tfp or sklearn-fork)
- Support L1 regularization

## Nice to have features

- Support Tweedie
- Ability to specify custom weights for parameters for the L1 penalty
- Ability to specify custom weight matrix for parameters for the L2 penalty (generalized Tikhonov regularization)

## Which packages/implementations exist?

- glmnet (R-Package, Fortran implementation, GPL-2, no Tweedie, no gamma)
https://glmnet.stanford.edu/

- pyglmnet (python only, no support of sample weights, no Tweedie, otherwise looks fairly feature-rich, under active development) https://github.com/glm-tools/pyglmnet

- python-glmnet (python bindings to glmnet's Fortran code, not actively developed) https://github.com/civisanalytics/python-glmnet

- glmnet_python (used at wayfair, GPL licensed) https://github.com/bbalasub1/glmnet_python

- scikit-learn fork (python only, has all the features we want, poor performance with sparse matrices, convergence trouble with lasso) https://github.com/scikit-learn/scikit-learn/pull/9405

- scikit-learn master (python only, only ridge, poor performnce with sparse matrices) https://github.com/scikit-learn/scikit-learn/pull/14300

- h2o (Java, python bindings, data needs to be copied from Python to Java all the time, convergence problems) http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html

- Tensorflow Probability (C++, python bindings, no sample weights) https://www.tensorflow.org/probability/api_docs/python/tfp/glm

- statsmodels (python only, very poor performance, lots of issues getting it to work) https://www.statsmodels.org/stable/glm.html
- [celer](https://github.com/mathurinm/celer): Fast solver for Lasso and sparse logistic regression.

## Other potentially useful reading material

- [1] Book: [Generalized Linear Models](http://www.utstat.toronto.edu/~brunner/oldclass/2201s11/readings/glmbook.pdf), McCullagh and Nelder
- [2] [Regularization Paths for GEneralized Linear Models via Coordinate Descent](http://statweb.stanford.edu/~jhf/ftp/glmnet.pdf). This is the "glmnet paper" that describes how the glment package works and the popular Coordinate Descent algorithm.
- [3] [Tensorflow GLM Tutorial](https://colab.research.google.com/github/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Generalized_Linear_Models.ipynb)
- [4] [Sklearn User guide](https://scikit-learn.org/dev/modules/linear_model.html#generalized-linear-regression)
- [5] [Tutorial on Insurance Claims Modelling with sklearn](https://github.com/lorentzenchr/Tutorial_freMTPL2/blob/master/glm_freMTPL2_example.ipynb)
- [6] [pyglmnet Tutorial](http://glm-tools.github.io/pyglmnet/tutorial.html)
- [7] [Improved GLMNET](https://www.csie.ntu.edu.tw/~cjlin/papers/l1_glmnet/long-glmnet.pdf)
- [8] [h2o Documentation on GLMs (fairly extensive)](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html)
- [9] [A Comparison of Optimization Methods and Software for Large-scale L1-regularized Linear Classification](https://www.csie.ntu.edu.tw/~cjlin/papers/l1.pdf), Yuan, Chang, Hsieh, and Lin 2010.
- [10] [An Improved GLMNET for L1-regularized Logistic Regression](http://www.jmlr.org/papers/volume13/yuan12a/yuan12a.pdf), Yuan, Ho, and Lin 2012. Is this the basis for the sklearn package's "improved" glmnet? Or tfp.glm.fit_sparse?
- [11] Worked example of Tweedie regression on insurance claims from [sklearn](https://github.com/scikit-learn/scikit-learn/blob/98054bc9a4416c49b26a3a253b9a7bef16a1e27b/examples/linear_model/plot_tweedie_regression_insurance_claims.py)
- [12] Write-up on [exposure](https://quantcoteam.slack.com/archives/C010MC8RHBK/p1585680163082400)
- [13] [Slides](https://www.groups.ma.tum.de/fileadmin/w00ccg/statistics/czado/lec2.pdf) on GLMs and IRLS from Claudia Czado